In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs

In [16]:
#initiate browser

executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

In [3]:
#scraping Mars News for latest headline title and body

mars_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(mars_url)

xpath_title = '//*[@id="page"]/div[2]/div/article/div/section/div/ul/li[1]/div/div/div[2]/a'
xpath_body = '//*[@id="page"]/div[2]/div/article/div/section/div/ul/li[1]/div/div/div[3]'

title_results = browser.find_by_xpath(xpath_title)
title_link = title_results[0]
recent_title = title_link.text

body_results = browser.find_by_xpath(xpath_body)
body_link = body_results[0]
recent_body = body_link.text

In [4]:
#confirm title text and body text
print(recent_title)
print(recent_body)

Third ASPIRE Test Confirms Mars 2020 Parachute a Go
The supersonic parachute that will handle the heaviest payload yet to the Red Planet – Mars 2020 rover – passes its final sounding rocket test with flying colors.


In [5]:
#crape for JPL Mars Space Images - Featured Image
pic_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
nasa_link = 'https://www.jpl.nasa.gov'

browser.visit(pic_url)
xpath_mars_link = '//*[@id="full_image"]'
xpath_mars_pic = '//*[@id="fancybox-lock"]/div/div[1]/img'

#store path of link and then click
mars_results = browser.find_by_xpath(xpath_mars_link)
mars_link = mars_results[0]
mars_link.click()

In [6]:
#read page with BeautifulSoup to obtain link
html = browser.html
soup = bs(html, 'html.parser')

#retrieve partial link
pic_link = soup.find("img", class_="fancybox-image")["src"]

#create link
mars_pic_link = nasa_link + pic_link
print(mars_pic_link)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18322_ip.jpg


In [11]:
#scrape for Mars Weather information

mars_weather_link = 'https://twitter.com/marswxreport?lang=en'

browser.visit(mars_weather_link)
xpath_mars_weather = '//*[@id="stream-item-tweet-1055479145478737921"]/div[1]/div[2]/div[2]/p'

weather_results = browser.find_by_xpath(xpath_mars_weather)
weather_body = weather_results[0]
mars_weather_body = weather_body.text

In [12]:
mars_weather_body

'Sol 2208 (2018-10-22), high -18C/0F, low -75C/-102F, pressure at 8.80 hPa, daylight 06:08-18:26'

In [13]:
#use Pandas to scrape Mars facts

mars_facts_url = 'https://space-facts.com/mars/'
mars_facts_table = pd.read_html(mars_facts_url)

In [14]:
#confirm the table

mars_facts_table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [15]:
#convert table into a dataframe
mars_facts_df = mars_facts_table[0]

#conver table into html
mars_facts_df.to_html('mars_facts.html')

In [3]:
#scrape for titles and pictures of Mars Hemispheres

pic_source = '//*[@id="wide-image"]/div/ul/li[2]/a'
mars_high_res_link = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
hemisphere_image_urls = []

#loop through the 4 links on the webpage
for x in range(1, 5):
    
    #get title of link
    browser.visit(mars_high_res_link)
    xpath = f'//*[@id="product-section"]/div[2]/div[{x}]/div/a/h3'
    results = browser.find_by_xpath(xpath)
    hemisphere_image_urls.append({
        "title": results.text
    })
    
    #get link of picture
    browser.visit(mars_high_res_link)
    xpath = f'//*[@id="product-section"]/div[2]/div[{x}]/div/a'
    results = browser.find_by_xpath(xpath)
    results.click()
    results = browser.find_by_xpath(pic_source)
    hemisphere_image_urls.append({
        "img_url": results["href"]
    })
    

In [7]:
#print dictionary to confirm results
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'},
 {'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'},
 {'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'},
 {'title': 'Valles Marineris Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]

In [45]:
#scrape for titles and pictures of Mars Hemispheres without xpath

mars_high_res_link = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
base_link = 'https://astrogeology.usgs.gov/'
hemisphere_image_urls = []

browser.visit(mars_high_res_link)
soup = bs(browser.html, 'lxml')
hemispheres = soup.find_all("div", class_="item")

for hemisphere in hemispheres:
    
    title = hemisphere.find("a", class_="itemLink").find("img")["alt"]
    title_edit = title[:-19]
    hemisphere_image_urls.append({
        "title": title_edit
    })
    
    hemi_link = hemisphere.find("a", class_="itemLink")
    image_link = hemi_link["href"]
    full_link = f'{base_link}{image_link}'
    hemisphere_image_urls.append({
        "img_url": full_link
    })

In [34]:

pic_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
nasa_link = 'https://www.jpl.nasa.gov'

browser.visit(pic_url)

xpath_mars_link = '//*[@id="full_image"]'
xpath_mars_pic = '//*[@id="fancybox-lock"]/div/div[1]/img'

#store path of link and then click
mars_results = browser.find_by_xpath(xpath_mars_link)

mars_link = mars_results[0]
mars_link.click()

#read page with BeautifulSoup to obtain link
html = browser.html
soup = bs(html, 'html.parser')

#retrieve partial link
#pic_link = soup.find("img", class_="fancybox-image")["src"]
pic_link = browser.find_by_xpath(xpath_mars_pic)


#create link
mars_pic_link = f'{pic_link[0]["src"]}'

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17551_ip.jpg'